In [6]:
import os
import zipfile
import dotenv
import subprocess
import pandas as pd
from typing import Dict, Any
import json
import yaml
import time
import glob
import hashlib
import shutil
import logging
import pandas as pd


from PIL import Image
from transformers import pipeline
import torch

config_train = "config/UrbanElementsReID_train.yml"  # "config/UAM_containers.yml"
#config_test = "config/UrbanElementsReID_test.yml"
config_test = "config/UrbanElementsReID_test_reduced.yml"
competition_name = "urban-reid-challenge"
submission_message = f"test trained submission"
experiment_id: int = int(time.time())


################ Probably nothing has to be modified from now on ################
with open(config_train, 'r') as f:
    hyperparams_train = yaml.load(f, Loader=yaml.BaseLoader)
with open(config_test, 'r') as f:
    hyperparams_test = yaml.load(f, Loader=yaml.BaseLoader)
model_path = os.path.join(hyperparams_train['LOG_ROOT'], hyperparams_train['LOG_NAME'])

assert dotenv.load_dotenv('../../.env')
assert os.getenv('KAGGLE_USERNAME')

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

logging.root.setLevel(logging.INFO)

if not torch.cuda.is_available():
    logging.warning("Where is your GPU dude?")

max_epoch = int(hyperparams_train['SOLVER']['MAX_EPOCHS'])
assert hyperparams_test['TEST']['WEIGHT'].split('/')[-1] == f'part_attention_vit_{max_epoch}.pth', 'not testing with the trained model...'

In [ ]:
!python update.py --config_file {config_test} --track {os.path.join(model_path, "track.txt")}

submission_file_name = os.path.join(model_path, "track_submission.csv")
assert os.path.exists(submission_file_name)
logging.info(f'Backuped everything to {experiment_id}')

!cp -r {model_path} {model_path}_backup_{experiment_id}

In [ ]:
assert os.path.exists(submission_file_name)
df = pd.read_csv(submission_file_name)
assert df.shape[1] == 2
assert df.shape[0] > 100

submission_message += f"; commit_hash: {subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode('ascii').strip()}"
submission_message += f"; hyperparameters_train_hash: {calculate_params_hash(hyperparams_train)}"
submission_message += f"; hyperparameters_test_hash: {calculate_params_hash(hyperparams_test)}"
submission_message += f"; experiment_ID: {experiment_id}"
logging.info(f'Submitting with message "{submission_message}"')

# Submit the file to the competition
# Uncomment only for actual submissions!
#api.competition_submit(submission_file_name, submission_message, competition_name)    